# Using dplyr with Adobe Analytics Data Feeds
So, after you've connected R to your query engine of choice, it's time to prepare your data feed for analysis. The below example shows how to connect to Adobe Cloud Platform Query Service which uses PostgreSQL, but you can use whatever it is that you need.

In [ ]:
require("RPostgreSQL")
library(dplyr)
library(dbplyr)

drv = dbDriver("PostgreSQL")
host = "my.database.host"
port = 80
usr = "myUserCredentials"
pw = rstudioapi::askForPassword("Password:")
con = dbConnect(
  drv, 
  dbname = "dbname=all sslmode=require", 
  host = host, 
  port = port, 
  user = usr, 
  password = pw
)

# Once authenticated, create a variable pointing to a db table:
my_data = tbl(con, "my_data_feed_table")

# Prepping a data feed for easier use
Once I'm logged in and connected to a table, I'm ready to start manipulating my data feed for analysis. The rest of the examples will use the sample data feed I've supplied so that you can easily reproduce what I've done on your local machine (or you can load the sample into your own query engine of choice).

First, we're going to do a bit of data clean up (this step is already done in the sample dataset). If you're doing this yourself, be sure to look at the column_headers file that comes with the datafeed to know which columns are what - you'll likely need to use the column names "V1", "V2" etc and rename them using a select verb.

In [6]:
data_feed_transformed = data_feed %>%
  mutate(
    # create a usable id from visid hi and lo:
    merged_visitor_ids = paste0(visitor_id_hi,"_",visitor_id_lo),
    
    # add a leading and trailing comma to the 
    # event list to make searching easier:
    fixed_event_list = ifelse(post_event_list == "", "", paste0(",",post_event_list,",")),
    
    # create a column for conversions - "1" in the event list
    orders = ifelse(grepl(",1,", fixed_event_list), 1, 0)
    
    # If using Adobe Query Service or Spark SQL, you'd do it like:
    # orders = ifelse(fixed_event_list %~% ",1,", 1, NA)
    
  ) %>%
  select(
    
    # datafeeds don't have column headers so refer to
    # the column headers file that comes with the 
    # data feed lookups to get this right
    
    visitor_id = merged_visitor_ids,
    visit_num,
    hit_time_gmt,
    event_list = fixed_event_list,
    post_campaign,
    orders,
    ip,
    user_id
    
  ) %>%
  mutate(
    # handy manipulation if your user id contains email addresses:
    # refer to your specific query engine documentation around
    # regex support
    
    # How to do this in Postgres
    # user_id = tolower(regexp_replace(user_id, "\\\\.", ""))
  )

# Notice that dbplyr doesn't actually execute the query until it needs to. The above code never touches your database until you run either "collect()" which returns the rows from the query, or perform another operation that requires the query to actually run like the "head()" below:

head(data_feed_transformed)

visitor_id,visit_num,hit_time_gmt,event_list,post_campaign,orders,ip,user_id
f73fe8ccf061718bf6de70f8ef6c1484_66360b670b0308e75eeeec6abeb15639,1,1517568611,",20,",campaign9,0,ip_address6351,NA
0198f59332915116862c6a1889538be2_0e303f38500e9f0f384451334a9e6bc3,1,1517569668,NA,NA,0,ip_address6545,user_id3795
457588336531e4208f3647e33fdb6028_a73fb9e74fee58fab7b64970aee739da,1,1517569678,",20,",campaign2233,0,ip_address5805,user_id1008
af5a0bbfd451ed2802265198f58688b4_278413a6033d4cea84136e6e495b5cfc,1,1517569662,",20,",campaign2240,0,ip_address6535,NA
af5a0bbfd451ed2802265198f58688b4_278413a6033d4cea84136e6e495b5cfc,1,1517569664,NA,campaign2240,0,ip_address6535,user_id2971
9741b5a62c07dbb2e606f5284e971685_d602a1427f25bab96a4dc0a063586603,1,1517569666,",2,20,",campaign89,0,ip_address6535,NA


# Adding classifications
Next let's apply a classification to the datafeed. In this case I'll classify the post_campaign to their marketing channel. In this example, we'll just use the classification file I supplied with the session materials - in your environment, you'll want to use a separate table you've uploaded.

In [9]:
classification = read.table(file="example_classification.tab", sep="\t", header = TRUE, stringsAsFactors = FALSE)
data_feed_with_class = data_feed_transformed %>%
  left_join(classification, by=c("post_campaign"))

head(data_feed_with_class)

visitor_id,visit_num,hit_time_gmt,event_list,post_campaign,orders,ip,user_id,channel
f73fe8ccf061718bf6de70f8ef6c1484_66360b670b0308e75eeeec6abeb15639,1,1517568611,",20,",campaign9,0,ip_address6351,NA,Affiliates
0198f59332915116862c6a1889538be2_0e303f38500e9f0f384451334a9e6bc3,1,1517569668,NA,NA,0,ip_address6545,user_id3795,NA
457588336531e4208f3647e33fdb6028_a73fb9e74fee58fab7b64970aee739da,1,1517569678,",20,",campaign2233,0,ip_address5805,user_id1008,Other Referrals
af5a0bbfd451ed2802265198f58688b4_278413a6033d4cea84136e6e495b5cfc,1,1517569662,",20,",campaign2240,0,ip_address6535,NA,Other Referrals
af5a0bbfd451ed2802265198f58688b4_278413a6033d4cea84136e6e495b5cfc,1,1517569664,NA,campaign2240,0,ip_address6535,user_id2971,Other Referrals
9741b5a62c07dbb2e606f5284e971685_d602a1427f25bab96a4dc0a063586603,1,1517569666,",2,20,",campaign89,0,ip_address6535,NA,Other Referrals


# De-attributing the data feed
Last, for attribution that we're going to do later, we need to "de-last-touch-ify" the data feed for the channels column. If you're using the "post" columns from the data feed, chances are you are looking at the last touch version of that channel written in to the data (we persist values from row to row as we collect the data). To undo that, we'll need to inspect the "campaign instance" event (the number "20") in the event list, and only keep the channel values on rows with that instance event.

In [10]:
data_feed_ready = data_feed_with_class %>%
  mutate(
    # Running locally on your laptop, use:
    non_attributed_channel = ifelse(grepl(",20,", event_list), channel, NA)
      
    # If using PostgreSQL, you'd do it like:
    #non_attributed_channel = ifelse(post_event_list ~ ",20,", channel, NA)
    
    # Or if using Spark SQL via sparklyr
    #non_attributed_channel = ifelse(post_event_list %regexp% ",20,", channel, NA)
  )

head(data_feed_ready)

visitor_id,visit_num,hit_time_gmt,event_list,post_campaign,orders,ip,user_id,channel,non_attributed_channel
f73fe8ccf061718bf6de70f8ef6c1484_66360b670b0308e75eeeec6abeb15639,1,1517568611,",20,",campaign9,0,ip_address6351,NA,Affiliates,Affiliates
0198f59332915116862c6a1889538be2_0e303f38500e9f0f384451334a9e6bc3,1,1517569668,NA,NA,0,ip_address6545,user_id3795,NA,NA
457588336531e4208f3647e33fdb6028_a73fb9e74fee58fab7b64970aee739da,1,1517569678,",20,",campaign2233,0,ip_address5805,user_id1008,Other Referrals,Other Referrals
af5a0bbfd451ed2802265198f58688b4_278413a6033d4cea84136e6e495b5cfc,1,1517569662,",20,",campaign2240,0,ip_address6535,NA,Other Referrals,Other Referrals
af5a0bbfd451ed2802265198f58688b4_278413a6033d4cea84136e6e495b5cfc,1,1517569664,NA,campaign2240,0,ip_address6535,user_id2971,Other Referrals,NA
9741b5a62c07dbb2e606f5284e971685_d602a1427f25bab96a4dc0a063586603,1,1517569666,",2,20,",campaign89,0,ip_address6535,NA,Other Referrals,Other Referrals


Awesome, now we're ready to go! Next up, creating a cross-device ID for analysis.